In [2]:
#Importing all the library used in the notebooks
using Profile
using ProfileView
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

### Versione iniziale di frag_face

In [3]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [4]:
V = Float64[
    0 0 0; 0 1 0;
    1 1 0; 1 0 0;
    0 0 1; 0 1 1;
    1 1 1; 1 0 1
]

8×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  1.0  0.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  1.0
 1.0  1.0  1.0
 1.0  0.0  1.0

In [5]:
EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
])

12×8 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
 -1   1   ⋅   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅  -1   1   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅   ⋅  -1   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   ⋅   1   ⋅   ⋅  ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1   ⋅  ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  ⋅
  ⋅   ⋅   ⋅  -1   ⋅   ⋅   ⋅  1
  ⋅   ⋅   ⋅   ⋅  -1   1   ⋅  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅  -1   1  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1
  ⋅   ⋅   ⋅   ⋅  -1   ⋅   ⋅  1

In [6]:
FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

`@btime` riporta i risultati come `@time`, mostrando le allocazioni ed il tempo minimo trascorso in una singola chiamata di funzione durante l'esecuzione della funzione su più campionamenti. Useremo `@btime` nella maggior parte dei nostri esempi.

In [7]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

  356.700 μs (4870 allocations: 265.34 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 1, 2, 2, 3, 3, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, 1, -1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, 1, 1, -1], 1, 4))

Utilizzando la macro `@code_warntype` si individuano molte variabili assegnate al tipo `Any`. Questo significa essenzialmente che ci sarà un'allocazione per la posizione della memoria e l'indirezione al valore effettivo durante il calcolo. 

In [8]:
@code_warntype frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

MethodInstance for frag_face(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face(V, EV, FE, sp_idx, sigma) in Main at c:\Users\Valerio\OneDrive - Universita degli Studi Roma Tre\CP\LAR TGW 3D\notebooks\frag_face.ipynb:1
Arguments
  #self#::Core.Const(frag_face)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Any
  @_8::Union{Nothing, Tuple{Int64, Nothing}}
  nvsize::Any
  nFE::Any
  nEV::Any
  nV::Any
  tV::Any
  M::Any
  sEV::Any
  sV::Any
  sigmavs::Vector{Int64}
  vs_num::Int64
  @_19::Any
  @_20::Any
  i::Int64
  tmpEV::Any
  tmpV::Any
Body::Tuple{Any, Any, Any}
1 ─       Core.NewvarNode(:(@_7))
│         Core.NewvarNode(:(nvsize))
│         Core.NewvarNode(:(nFE))
│         Core.NewvarNode(:(nEV))
│         Core.NewvarNode(:(nV))
│         (vs_num = Main.size(V, 1))
│   %7  = (sigma:sigma)::UnitRange{Int64}
│   %8

Utilizzando la macro `@profile` si immagazzinano le informazioni relative alla funzione che successivamente vengono mostrate graficamente. Tramite il grafico riusciamo a dare una misurazione temporale di ogni singola funzione. La larghezza delle barre mostra il tempo trascorso in ogni locazione di chiamata, mentre la gerarchia di chiamata è rappresentata dalle varie altezze del grafico.

In [9]:
Profile.clear()
Profile.init(delay=.000000001)
@profile for _ in 1:200
    @profile frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [10]:
ProfileView.view()

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

# Ottimizzazione



Possiamo assegnare alle variabili locali un tipo deterministico per rimuovere il tipo `Any`.
Inoltre si possono creare delle viste degli array, con la macro `@views`, che permettono di accedere ai valori dell'array senza dover effettuare una copia.

Si nota come utilizzando le macro `@views` e `@async` la funzione frag_face ottiene un nuovo tempo minimo con un decremento di circa 20% rispetto alla funzione originale.

In [11]:
function frag_face2(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
    @views tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    @views sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    @views nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face2 (generic function with 1 method)

### Tempi minimi ottenuti da `@btime` con le diverse macro:

`@async` = 292.500 μs (3873 allocations: 218.65 KiB)

`Threads.@spawn` = 305.200 μs (4093 allocations: 230.38 KiB)

`@simd` = 383.000 μs (4996 allocations: 269.99 KiB)

`@inbounds` = 377.700 μs (4996 allocations: 269.99 KiB)

In [12]:
@btime frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

  288.000 μs (3873 allocations: 218.65 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 1, 2, 2, 3, 3, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, 1, -1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, 1, 1, -1], 1, 4))

In [13]:
@code_warntype frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

MethodInstance for frag_face2(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face2(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, sp_idx::Vector{Int64}, sigma::Int64) in Main at c:\Users\Valerio\OneDrive - Universita degli Studi Roma Tre\CP\LAR TGW 3D\notebooks\frag_face.ipynb:1
Arguments
  #self#::Core.Const(frag_face2)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Any
  nvsize::Int64
  nFE::SparseMatrixCSC{Int8, Int64}
  nEV::SparseMatrixCSC{Int8, Int64}
  nV::Matrix{Float64}
  tV::Matrix{Float64}
  M::Matrix{Float64}
  sEV@_14::Core.Box
  sV@_15::Core.Box
  sigmavs::Vector{Int64}
  vs_num::Int64
  #9::var"#9#10"{SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, Vector{Int64}, Int64, Matrix{Float64}}
  task::Task
  sV@_20::Union{}
  sV@_21::Union{}
  sV@_22::Union{

sEV@_23
15 ┄ %113 = Core.getfield(sEV@_14, :contents)::Any
│    %114 = Core.apply_type(Main.SparseMatrixCSC, Main.Int8, Main.Int64)::Core.Const(SparseMatrixCSC{Int8, Int64})
│    %115 = Core.typeassert(%113, %114)::SparseMatrixCSC{Int8, Int64}
│    %116 = Main.length(sigmavs)::Int64
│    %117 = Main.ones(Main.Int8, %116)::Vector{Int8}
│    %118 = Main.sparsevec(%117)::SparseVector{Int8, Int64}
│    %119 = (%99)(%107, %115, %118)::Any
│    %120 = Base.indexed_iterate(%119, 1)::Any
│    %121 = Core.getfield(%120, 1)::Any
│           (@_7 = Core.getfield(%120, 2))
│    %123 = Base.indexed_iterate(%119, 2, @_7)::Any
│    %124 = Core.getfield(%123, 1)::Any
│           (@_7 = Core.getfield(%123, 2))
│    %126 = Base.indexed_iterate(%119, 3, @_7)::Any
│    %127 = Core.getfield(%126, 1)::Any
│    %128 = Core.apply_type(Main.Matrix, Main.Float64)::Core.Const(Matrix{Float64})
│    %129 = Base.convert(%128, %121)::Any
│           (nV = Core.typeassert(%129, %128))
│    %131 = Core.apply_type(Main

In [14]:
Profile.clear()
Profile.init(delay=.000000001)
@profile for _ in 1:200
    frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [15]:
ProfileView.view()

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [16]:
if(frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2) == frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2))
    print("Uguali")
else
    print("Diversi")
end

Uguali

In [17]:
@benchmark frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

BenchmarkTools.Trial: 8564 samples with 1 evaluation.
 Range (min … max):  364.500 μs … 28.761 ms  ┊ GC (min … max):  0.00% … 93.91%
 Time  (median):     427.700 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   580.318 μs ±  1.258 ms  ┊ GC (mean ± σ):  15.37% ±  7.25%

  ▁▆██▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▂▂▁▁▂▂▂▂▂▂▁▁▁▁▂▁                  ▂
  ████████████████████████████████████████████████▇▇█▇▇▆▅▆▄▅▅▄ █
  364 μs        Histogram: log(frequency) by time       993 μs <

 Memory estimate: 265.71 KiB, allocs estimate: 4874.

In [18]:
@benchmark frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

BenchmarkTools.Trial: 9673 samples with 1 evaluation.
 Range (min … max):  282.100 μs … 35.334 ms  ┊ GC (min … max):  0.00% … 97.93%
 Time  (median):     354.600 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   512.921 μs ±  1.559 ms  ┊ GC (mean ± σ):  20.08% ±  6.64%

    ██▂                                                         
  ▁▃███▇▅▄▄▅▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  282 μs          Histogram: frequency by time          850 μs <

 Memory estimate: 218.65 KiB, allocs estimate: 3873.